In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import pickle
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import dataloader as dl

In [3]:
train = dl.MyDataset(root_dir = "./",transform=transforms.Compose([transforms.ToTensor()]))
trainloader = torch.utils.data.DataLoader(train, batch_size=4,
                                      shuffle=True, num_workers=2)

In [20]:
class detector(nn.Module):
    def __init__(self):
        super(detector, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=1, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),  
            nn.ReLU(True)
        )
        self.pool1 = nn.MaxPool2d(2, stride=2)  
    
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, stride=1, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(32, 32, 3, stride=1, padding=1),  
            nn.ReLU(True)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=1, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(64, 64, 3, stride=1, padding=1),  
            nn.ReLU(True)
        )
        self.pool2 = nn.MaxPool2d(4,stride=4)
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(64,64,(7, 9),stride=1),
            nn.ReLU(True),
            nn.Conv2d(64,6,1,stride=1),
            nn.ReLU(True)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool1(x)  
        x = self.conv3(x)
        x = self.pool2(x)
        x = self.conv4(x)
        return x
    
model = detector().cuda(1)
def compute_loss(output,label):
    label = label.float()
    l_coord = 5
    l_nobj = 0.5
    sw_obj = label[:,0,:,:]
    sw_nobj = (label[:,0,:,:]==0).float()
    
    pred_loss_obj = torch.sum(nn.MSELoss(reduce=False)(output[:,0,:,:],label[:,0,:,:])*sw_obj)
    pred_loss_nobj = l_nobj*torch.sum(nn.MSELoss(reduce=False)(output[:,0,:,:],label[:,0,:,:])*sw_nobj)
    box_xy_loss = l_coord*torch.sum(nn.MSELoss(reduce=False)(output[:,1:3,:,:],label[:,1:3,:,:])*sw_obj)
    box_hw_loss = l_coord*torch.sum(nn.MSELoss(reduce=False)(torch.sqrt(output[:,3:5,:,:]),torch.sqrt(label[:,3:5,:,:]))*sw_obj)
    class_loss = torch.sum(nn.MSELoss(reduce=False)(output[:,5,:,:],label[:,5,:,:])*sw_obj)
    loss = pred_loss_obj + pred_loss_nobj + box_xy_loss + box_hw_loss + class_loss
    return loss

optimizer = torch.optim.Adam(model.parameters(), lr=0.01,
                             weight_decay=1e-5)

In [5]:
num_epochs = 100
for epoch in range(num_epochs):
    for data in trainloader:
        img, label = data
        img = Variable(img).cuda(1)
        label = Variable(label).cuda(1)

        # ===================forward=====================
        output = model(img)
        loss = compute_loss(output, label)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
      .format(epoch+1, 20, loss.data[0]))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch [1/20], loss:271.8044
epoch [2/20], loss:253.5497
epoch [3/20], loss:201.0186
epoch [4/20], loss:248.9474
epoch [5/20], loss:143.0202
epoch [6/20], loss:315.2001
epoch [7/20], loss:115.5266
epoch [8/20], loss:153.8384
epoch [9/20], loss:95.4768
epoch [10/20], loss:231.2339
epoch [11/20], loss:240.8128
epoch [12/20], loss:177.6854
epoch [13/20], loss:139.4938
epoch [14/20], loss:172.2450
epoch [15/20], loss:256.7700
epoch [16/20], loss:234.8800
epoch [17/20], loss:276.9067
epoch [18/20], loss:199.3510
epoch [19/20], loss:176.0418
epoch [20/20], loss:226.4730
epoch [21/20], loss:316.9384
epoch [22/20], loss:185.9048


Process Process-46:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python2.7/dist-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "dataloader.py", line 27, in __getitem__
    image = cv2.imread(self.img_path + self.image_list[idx])
KeyboardInterrupt
Process Process-45:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python2.7/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python2.7/multiproce

KeyboardInterrupt: 

torch.Size([4, 6, 24, 32])



(0 ,0 ,.,.) = 
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
           ...             ⋱             ...          
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
     ⋮ 

(1 ,0 ,.,.) = 
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
           ...             ⋱             ...          
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
     ⋮ 

(2 ,0 ,.,.) = 
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
  0.5000  0.5000  0.5000  ...   0.5000  0.5000  0.5000
 